In [1]:
import pandas as pd

In [2]:
data = pd.read_parquet("yellow_tripdata_2022-01.parquet")

In [4]:
data.shape

(2463931, 19)

In [5]:
data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [7]:
data.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [3]:
pd.options.display.float_format = str
data["duration"] = ((data.tpep_dropoff_datetime - data.tpep_pickup_datetime).dt.total_seconds()/60)

In [4]:
import numpy as np
data.duration.std()

46.44530513776499

In [54]:
data.duration.describe()

count            2463931.0
mean    14.212202918831741
std      46.44530513776499
min                -3442.4
25%      6.316666666666666
50%     10.183333333333334
75%     16.166666666666668
max      8513.183333333332
Name: duration, dtype: float64

In [5]:
mask = ((data.duration >= 1) & (data.duration<=60))

In [6]:
mask.value_counts(normalize=True)

True      0.9827547930522406
False   0.017245206947759495
Name: duration, dtype: float64

In [7]:
data = data.loc[mask]

In [11]:
from sklearn.feature_extraction import DictVectorizer

In [8]:
data.PULocationID = data.PULocationID.astype("str")
data.DOLocationID = data.DOLocationID.astype("str")

In [9]:
features = ["PULocationID","DOLocationID"]

In [10]:
dict_locations = data[features].to_dict(orient="records")

In [12]:
vectorizer = DictVectorizer()
feature_matrix = vectorizer.fit_transform(dict_locations)

In [13]:
feature_matrix.shape

(2421440, 515)

In [ ]:
# https://datascience.stackexchange.com/questions/9443/when-to-use-one-hot-encoding-vs-labelencoder-vs-dictvectorizor


In [92]:
data.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee', 'duration'],
      dtype='object')

In [14]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()data["duration"] = ((data.tpep_dropoff_datetime - data.tpep_pickup_datetime).dt.total_seconds()/60)

LinearRegression()

In [15]:
from sklearn.metrics import mean_squared_error

pred = lr.predict(feature_matrix)
mean_squared_error(data.duration,pred,squared=False)

6.986191073878161

In [29]:
data_eval = pd.read_parquet("yellow_tripdata_2022-02.parquet")

In [30]:
data_eval["duration"] = ((data_eval.tpep_dropoff_datetime - data_eval.tpep_pickup_datetime).dt.total_seconds()/60)

In [37]:
mask = ((data_eval.duration >= 1) & (data_eval.duration<=60))

data_eval = data_eval.loc[mask]

In [38]:
data_eval.PULocationID = data_eval.PULocationID.astype("str")
data_eval.DOLocationID = data_eval.DOLocationID.astype("str")

In [39]:
eval_dict = data_eval[features].to_dict(orient="records")

In [40]:
eval_fm = vectorizer.transform(eval_dict)

In [41]:
eval_fm.shape

(2918187, 515)

In [42]:
pred = lr.predict(eval_fm)
mean_squared_error(data_eval.duration,pred,squared=False)

7.786412300747138